# 5300 Final: Solving Double Pendulums with Lagrange

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
from scipy.integrate import odeint, solve_ivp
from matplotlib import animation, rc
from IPython.display import HTML

Here we solve the Lagrange equations for the motion of a double pendulum and illustrate that the motion is chaotic. We first write down the Lagrangian and find Lagrange's equations.

In [ ]:
Image(url="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Double-Pendulum.svg/255px-Double-Pendulum.svg.png")

Write down the Lagrangian:

$\mathcal{L} = \frac{1}{2}m_1L_1^2\dot{\theta_1}^2 + \frac{1}{2}m_2[L_1^2\dot{\theta_1}^2 + L_2^2\dot{\theta_2}^2 + 2L_1L_2\dot{\theta_1}\dot{\theta_2}cos(\theta_1-\theta_2)] + g[m_1L_1cos(\theta_1) + m_2(L_1cos(\theta_1)+L_2cos(\theta_2))] $

Now calculate derivatives and write down Lagrangian equations for $\theta_1$ and $\theta_2$:

$\frac{\partial\mathcal{L}}{\partial\theta_1} = \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial\dot{\theta_1}} \implies  (m_1+m_2)[L_1\ddot{\theta_1} + gsin(\theta_1)] + m_2L_2[\ddot{\theta_2}cos(\theta_1-\theta_2) + \dot{\theta_2}^2sin(\theta_1-\theta_2)] = 0$ 

$\frac{\partial\mathcal{L}}{\partial\theta_2} = \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial\dot{\theta_2}} \implies  L_2\ddot{\theta_2} + L_1\ddot{\theta_1}cos(\theta_1-\theta_2) + gsin(\theta_2) - L_1\dot{\theta_1^2}sin(\theta_1 - \theta_2) = 0$ 

For solve_ivp, we need first order differential equations, so we will make subsitutions and rearrange to get 4 first order equations in place of our two second order equations: 

$z_1 = \dot{\theta_1}$ 

$z_2 = \dot{\theta_2}$

$\dot{z_1} = \frac{m_2gsin(\theta_2)cos(\theta_1-\theta_2) - (m_1+m_2)gsin(\theta_1)-m_2sin(\theta_1-\theta_2)[L_1z_1^2cos(\theta_1-\theta_2) + L_2z_2^2]}{L_1(m_1+m_2sin^2(\theta_1-\theta_2))}$

$\dot{z_2} = \frac{(m_1+m_2)[-gsin(\theta_2) + gsin(\theta_1)cos(\theta_1-\theta_2) + L_1z_1^2sin(\theta_1-\theta_2)] + m_2L_2z_2^2sin(\theta_1-\theta_2)cos(\theta_1-\theta_2)}{L_2(m_1+m_2sin^2(\theta_1-\theta_2))}$







## Double Pendulum Class

In [ ]:
class DoublePendulum():
    """
    Pendulum class implements the parameters and Lagrange's equations for 
     a double pendulum
     
    Parameters
    ----------
    L1, L2 : float
        length of pendulum arms
    g : float
        gravitational acceleration at the earth's surface
    m1, m2: float
        masses of the first and second pendulum bobs 

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1=1., L2=1., m1=1., m2=1., g=1.
                ):
        self.L1 = L1
        self.L2 = L2
        self.m1 = m1
        self.m2 = m2

        self.g = g
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            A 4-component vector with y[0] = theta1(t), y[1] = theta2(t), y[2] = dtheta1/dt, y[3]=dtheta2/dt
            
        Returns
        -------
        List with solution for each coordinate
        """
        m1 = self.m1
        m2 =self.m2
        L1 = self.L1
        L2 = self.L2
        g = self.g
        theta1 = y[0]
        theta2 =y[1]
        z1 = y[2]
        z2 = y[3]
        
        z1_dot = ((m2*g*np.sin(theta2)*np.cos(theta1-theta2)) - (g*np.sin(theta1)*(m1+m2)) - m2*np.sin(theta1-theta2) \
                *((L1*z1**2)*np.cos(theta1-theta2) + L2*z2**2))/(L1*(m1+m2*(np.sin(theta1-theta2))**2))
        z2_dot = ((m1+m2)*(-g*np.sin(theta2) + g*np.sin(theta1)*np.cos(theta1-theta2) + L1*z1**2*np.sin(theta1-theta2)) + \
                 m2*L2*z2**2*np.sin(theta1-theta2)*np.cos(theta1-theta2)  )/(L1*(m1+m2*(np.sin(theta1-theta2))**2))
        
        return [y[2], y[3], z1_dot, z2_dot]
    
    def solve_ode(self, t_pts, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0,  
                  abserr=1.0e-10, relerr=1.0e-10):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [theta1_0, theta2_0, theta1_dot_0, theta2_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        theta1, theta2, theta1_dot, theta2_dot = solution.y

        return theta1, theta2, theta1_dot, theta2_dot

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

## Test Cases

Now that we have our class and helper functions set up, let's try some initial conditions and see what behavior we get:

In [ ]:
# Labels for individual plot axes
theta_vs_time_labels = (r'$t$', r'$\theta(t)$')
theta1_vs_time_labels = (r'$t$', r'$\theta_1(t)$')
theta2_vs_time_labels = (r'$t$', r'$\theta_2(t)$')
theta1_dot_vs_time_labels = (r'$t$', r'$d\theta_1/dt(t)$')
theta2_dot_vs_time_labels = (r'$t$', r'$d\theta_2/dt(t)$')

state_space1_labels = (r'$\theta_1$', r'$d\theta_1/dt$')
state_space2_labels = (r'$\theta_2$', r'$d\theta_2/dt$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 50.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

L1 = 1.
L2 = 1.
m1 = 1.
m2 = 1.
g = 1.

# Instantiate a pendulum 
p1 = DoublePendulum(L1 = L1, L2=L2, m1 = m1, m2 = m2, g=g)


In [ ]:
#Initial conditions
theta1_0_l = [np.pi/2., np.pi/2., np.pi/2.]
theta2_0_l = [np.pi, 0., np.pi/2.]
theta1_dot_0_l = [0., 0., 0.]
theta2_dot_0_l = [0., np.pi/3., 0.]


# start the plot!
fig = plt.figure(figsize=(8,10))
overall_title = rf'Double pendulum from Lagrangian:  ' + \
                rf'L1 = L2 = m1 = m2 = g = 1'
fig.suptitle(overall_title, va='baseline')

nrow = len(theta1_0_l)

#Solve and plot for each initial condition
for i in range(0, nrow):
    
    theta1_0 = theta1_0_l[i]
    theta2_0 = theta2_0_l[i]
    theta1_dot_0 = theta1_dot_0_l[i]
    theta2_dot_0 = theta2_dot_0_l[i]
    
    theta1, theta2, theta1_dot, theta2_dot = p1.solve_ode(t_pts, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0)
    
    # first plot: theta1 plot 
    ax_a = fig.add_subplot(nrow,1,i+1)                  

    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(t_pts[start : stop], theta1[start : stop], 
                axis_labels=theta_vs_time_labels, 
                color='blue',
                label=rf'$\theta_1(t)$', 
                title=rf'$\theta_1(0)={theta1_0:.3}, \theta_2(0)={theta2_0:.3}, $' + \
                rf'$\dot{{\theta_1}}(0)={theta1_dot_0:.3}, \dot{{\theta_2}}(0)={theta2_dot_0:.3}, $',
                ax=ax_a)       
    plot_y_vs_x(t_pts[start : stop], theta2[start : stop], 
                axis_labels=None, 
                color='red',
                label=rf'$\theta_2(t)$', 
                title=None, 
                ax=ax_a)    

fig.tight_layout()


The first two look similar to what Dr. Furnstahl got, so we feel good that our code is working. Note on the third we start with the pendulum bobs in a straight line and perpendicular to the vertical. Very naively, one might expect the pendulum bobs to stay in line and and act like a single pendulum, but this clearly does not happen (this would mean $\theta_2= \theta_1$ at all times), although they stay somewhat in sync for t<20. Note we don't expect this to work, as in this case both pendulum bobs would both be subject only to the force of gravity and a centripetal force from the connecting rods. But in this case the second bob would oscillate with frequency $\sqrt{\frac{g}{L_1+L_2}}$ and the first bob would oscillate with period $\sqrt{\frac{g}{L_1}}$. But this means they oscillate with different periods and will not stay in sync!

## Animate!

Let's go ahead and animate now so we can see what the motion looks like:

In [ ]:
# The dpi (dots-per-inch) setting will affect the resolution and how large
#  the plots appear on screen and printed.  So you may want/need to adjust 
#  the figsize when creating the figure.
plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 10
plt.rcParams.update({'font.size': font_size})

In [ ]:
class AnimationPendulumPlot():
    """
    AnimationPlot class uses matplotlib.animation.FuncAnimation to animate
     the dynamics of an oscillator.  This includes a simple time dependence
     graph, a state space graph with Poincare map, and a physical model.
     
     We'll start with a pendulum and then generalize later.
     
    Parameters
    ----------
    phi_vs_t : boolean
        If True, plot phi(t) vs. t
    
    phi_dot_vs_t : boolean
        If True, plot phi_dot(t) vs. t
    
    state_space : boolean
        If True, plot phi_dot(t) s. phi(t)
    
    physics_pend : boolean
        If True, draw the pendulum at phi(t) vs. t


    Methods
    -------
    plot_setup
    
    t_pts_init
    
    add_pendulum
    
    animate_pendulum
    
    plot_setup
    
    start_animation
    """
    def __init__(self, theta_vs_t=True, theta_dot_vs_t=False,
                physical_pend=True):
        self.theta1_list = []
        self.theta2_list = []
        self.theta1_dot_list = []
        self.theta2_dot_list = []
        self.theta_vs_t = theta_vs_t
        self.length = 0.8
        self.line_colors = ['blue', 'red']
        self.pt_colors = ['blue', 'red']
        self.theta_align = ['left', 'right']
                  
    def t_pts_init(self, t_start=0., t_end=100., delta_t=0.01): 
        """Create the array of time points for the full iteration"""
        self.t_start = t_start
        self.t_end = t_end
        self.delta_t = delta_t
        self.t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

    def add_pendulum(self, pend, theta1_0=0., theta2_0 = 0., theta1_dot_0=0., theta2_dot_0=0.):
        """Add a pendulum to be plotted as a class instance of Pendulum
            along with initial conditions.  So it knows all of the parameters
            as well through the Pendulum class.
        """
        self.pend = pend
        self.L1 = self.pend.L1
        self.L2 = self.pend.L2
        theta1, theta2, theta1_dot, theta2_dot = pend.solve_ode(self.t_pts, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0)
        self.theta1_list.append(theta1)
        self.theta2_list.append(theta2)
        self.theta1_dot_list.append(theta1_dot)
        self.theta2_dot_list.append(theta2_dot)

    def plot_setup(self, plot_start, plot_end):
        """Set up the plots to be displayed. """

            # start the plot!
#         overall_title = 'Parameters:  ' + \
#                         rf' $\omega = {omega_ext:.2f},$' + \
#                         rf' $\gamma = {gamma_ext:.3f},$' + \
#                         rf' $\omega_0 = {omega_0:.2f},$' + \
#                         rf' $\beta = {beta:.2f},$' + \
#                         rf'  $\phi_0 = {phi_0:.2f},$' + \
#                         rf' $\dot\phi_0 = {phi_dot_0:.2f}$' + \
#                         '\n'     # \n means a new line (adds some space here)
#         self.fig = plt.figure(figsize=(10,3.3), num='Pendulum Plots')
#         self.fig.suptitle(overall_title, va='top')

        # Labels for individual plot axes
        theta_vs_time_labels = (r'$t$', r'$\theta(t)$')
        theta_dot_vs_time_labels = (r'$t$', r'$d\theta/dt(t)$')
        state_space1_labels = (r'$\theta_1$', r'$d\theta_1/dt$')
        state_space2_labels = (r'$\theta_2$', r'$d\theta_2/dt$')

        self.fig = plt.figure(figsize=(10, 2.8), num='Pendulum animation')

        
        num_plots = 2
        if not self.theta_vs_t:
            num_plots = 1
        
        if self.theta_vs_t:
            self.ax_1 = self.fig.add_subplot(1,num_plots,1)        
            self.ax_1.set_xlabel(r'$t$')
            self.ax_1.set_ylabel(r'$\theta(t)$')
            self.line_1 = []
            self.line_1_2 = []

            self.pt_1 = []
            self.pt_1_2 = []

        self.ax_2 = self.fig.add_subplot(1,num_plots,num_plots)
        self.ax_2.set_aspect(1)   # aspect ratio 1 subplot
        #self.ax_2.set_origin(0.)   # origin in the middle
        #self.ax_2.set_theta_zero_location('S')  # phi=0 at the bottom
        pad = 0.25
        self.ax_2.set_ylim(-self.L1 - self.L2 - pad,self.L1+self.L2 + pad)
        self.ax_2.set_xlim(-self.L1 - self.L2-pad,self.L1+self.L2+pad)  
        self.ax_2.grid(False)   # no longitude/lattitude lines
        self.ax_2.set_xticklabels([])   # turn off angle labels
        self.ax_2.set_yticklabels([])    # turn off radial labels
        self.ax_2.axis('off') # no circular border
        self.line_2 = []
        self.line_2_2 = []
        self.pt_2 = []
        self.pt_2_2 = []

        self.theta_text = []

       
        
        # plot new arrays from start to stop
        self.start, self.stop = start_stop_indices(self.t_pts, plot_start, 
                                                   plot_end)
        self.t_pts_plt = self.t_pts[self.start : self.stop]
        self.theta1_plt_list = []
        self.theta2_plt_list = []
        self.theta1_dot_plt_list = []
        self.theta2_dot_plt_list = []

        for i, (theta1, theta2, theta1_dot, theta2_dot) in enumerate(zip(self.theta1_list,self.theta2_list, 
                                                                         self.theta1_dot_list, self.theta2_dot_list)):
            theta1_plt = theta1[self.start : self.stop]
            self.theta1_plt_list.append(theta1_plt)
            
            theta2_plt = theta2[self.start : self.stop]
            self.theta2_plt_list.append(theta2_plt)
            
            theta1_dot_plt = theta1_dot[self.start : self.stop]
            self.theta1_dot_plt_list.append(theta1_dot_plt)

            theta2_dot_plt = theta2_dot[self.start : self.stop]
            self.theta2_dot_plt_list.append(theta2_dot_plt)    
            
            if self.theta_vs_t:
                line_1, = self.ax_1.plot(self.t_pts_plt, theta1_plt, 
                                          color=self.line_colors[0],  label=rf'$\theta_1$')

                line_1_2, = self.ax_1.plot(self.t_pts_plt, theta2_plt, 
                              color=self.line_colors[1], label=rf'$\theta_2$')
                self.ax_1.legend()
                self.line_1.append(line_1)
                self.line_1_2.append(line_1_2)

                pt_1, = self.ax_1.plot(self.t_pts_plt[0], theta1_plt[0], 
                                        'o', color=self.pt_colors[0])
                self.pt_1.append(pt_1)

                pt_1_2, = self.ax_1.plot(self.t_pts_plt[0], theta2_plt[0], 
                                        'o', color=self.pt_colors[1])
                self.pt_1_2.append(pt_1_2)
            
            self.ax_2.plot(0, 0, color='black', marker='o', markersize=5)
            
            x_pts_2 = np.array([0., self.L1*np.sin(theta1_plt[0])])
            if theta1_plt[0]!=0.:
                y_pts_2 = -(1./np.tan(theta1_plt[0]))*x_pts_2           
            else:
                y_pts_2 = [0., -self.L1]
                
            line_2, = self.ax_2.plot(x_pts_2, y_pts_2, 
                                      color=self.line_colors[i], lw=3)
            self.line_2.append(line_2)
            pt_2, = self.ax_2.plot(self.L1*np.sin(theta1_plt[0]), -self.L1*np.cos(theta1_plt[0]), 
                                   marker='o', markersize=15, 
                                   color=self.pt_colors[i])
            self.pt_2.append(pt_2)
            
            x_pts_2_2 = np.array([self.L1*np.sin(theta1_plt[0]), self.L1*np.sin(theta1_plt[0]) + self.L2*np.sin(theta2_plt[0])])
           
            if theta2_plt[0]!=0.:
                y_pts_2_2 = -self.L1*np.cos(theta1_plt[0])-(1./np.tan(theta2_plt[0]))*(x_pts_2_2-self.L1*np.sin(theta1_plt[0]))           
            else:
                y_pts_2_2 = [-self.L1*np.cos(theta1_plt[0]), -self.L1*np.cos(theta1_plt[0])-self.L2 ]

            line_2_2, = self.ax_2.plot(x_pts_2_2, y_pts_2_2, 
                                      color=self.line_colors[i], lw=3)
            self.line_2_2.append(line_2_2)
            pt_2_2, = self.ax_2.plot(self.L1*np.sin(theta1_plt[0])+self.L2*np.sin(theta2_plt[0]), 
                                     -self.L1*np.cos(theta1_plt[0])-self.L2*np.cos(theta2_plt[0]), 
                                   marker='o', markersize=15, 
                                   color=self.pt_colors[i])
            self.pt_2_2.append(pt_2_2)      
                   
        
        
        self.fig.tight_layout()
        

    def animate_pendulum(self, i, t_pts_skip, theta1_skip_list,
                         theta1_dot_skip_list, theta2_skip_list, theta2_dot_skip_list):
        
        for index, (theta1_skip, theta1_dot_skip, theta2_skip, theta2_dot_skip) in \
                     enumerate(zip(theta1_skip_list, theta1_dot_skip_list, theta2_skip_list, theta2_dot_skip_list)):
            
            if self.theta_vs_t:
                self.pt_1[index].set_data(t_pts_skip[i], theta1_skip[i])
                self.pt_1_2[index].set_data(t_pts_skip[i], theta2_skip[i])

            x_pts_2 = np.array([0., self.L1*np.sin(theta1_skip[i])])
            if theta1_skip[i]!=0.:
                y_pts_2 = -(1./np.tan(theta1_skip[i]))*x_pts_2           
            else:
                y_pts_2 = [0., -self.L1]
            self.line_2[index].set_data(x_pts_2, y_pts_2)
            
            x_pts_2_2 = np.array([self.L1*np.sin(theta1_skip[i]), self.L1*np.sin(theta1_skip[i]) + self.L2*np.sin(theta2_skip[i])])
            if theta2_skip[i]!=0.:
                y_pts_2_2 = -self.L1*np.cos(theta1_skip[i])-(1./np.tan(theta2_skip[i]))*(x_pts_2_2-self.L1*np.sin(theta1_skip[i]))          
            else:
                y_pts_2_2 = [-self.L1*np.cos(theta1_skip[i]), -self.L1*np.cos(theta1_skip[i])-self.L2 ]

            self.line_2_2[index].set_data(x_pts_2_2, y_pts_2_2)

            self.pt_2[index].set_data(self.L1*np.sin(theta1_skip[i]), -self.L1*np.cos(theta1_skip[i]))
            self.pt_2_2[index].set_data(self.L1*np.sin(theta1_skip[i])+self.L2*np.sin(theta2_skip[i]), 
                                     -self.L1*np.cos(theta1_skip[i])-self.L2*np.cos(theta2_skip[i]))

        #return self.pt_1, self.pt_2, self.phi_text, self.pt_3
          
        
    def start_animation(self, skip=2, interval=25.):
        self.skip = skip          # skip between points in t_pts array
        self.interval = interval  # time between frames in milliseconds
        
        theta1_skip_list = []
        theta1_dot_skip_list = []
        theta2_skip_list = []
        theta2_dot_skip_list = []        

        for i, (theta1_plt, theta1_dot_plt, theta2_plt, theta2_dot_plt) in enumerate(zip(self.theta1_plt_list, 
                                        self.theta1_dot_plt_list, self.theta2_plt_list, self.theta2_dot_plt_list)):
            theta1_skip_list.append(theta1_plt[::self.skip])
            theta1_dot_skip_list.append(theta1_dot_plt[::self.skip])
            theta2_skip_list.append(theta2_plt[::self.skip])
            theta2_dot_skip_list.append(theta2_dot_plt[::self.skip])
                              
                
                
        t_pts_skip = self.t_pts_plt[::self.skip]                                   
        self.anim = animation.FuncAnimation(self.fig, self.animate_pendulum, 
                           fargs=(t_pts_skip,
                                  theta1_skip_list, theta1_dot_skip_list, theta2_skip_list, theta2_dot_skip_list
                                 ), 
                           init_func=None,
                           frames=len(t_pts_skip), 
                           interval=self.interval, 
                           blit=False, repeat=False,
                           save_count=0)

        #HTML(anim.to_jshtml())
        self.fig.show()


In [ ]:
# Create a pendulum animation instance. 
pendulum_anim = AnimationPendulumPlot(theta_vs_t=True, 
                                      physical_pend=True)

# Common plotting time (generate the full time here then use slices below)
t_start = 0.
t_end = 50.
delta_t = 0.01
pendulum_anim.t_pts_init(t_start, t_end, delta_t)

# INITIAL CONDITIONS
theta1_0 = np.pi/2.
theta2_0 = np.pi/2.
theta1_dot_0 = 0.
theta2_dot_0 = 0.


# Add a pendulum to the animation plots; this solves the differential
#  equation for the full t_pts array, generating phi and phi_dot internally. 
pendulum_anim.add_pendulum(p1, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0)
#pendulum_anim.add_pendulum(p1, theta1_0p, theta2_0, theta1_dot_0, theta2_dot_0)

In [ ]:
plot_start = 0.   # time to begin plotting
plot_end = 50.    # time to end plotting
pendulum_anim.plot_setup(plot_start, plot_end)

# Start the animation (adjust skip and interval for a smooth plot at a 
#  useful speed)
skip = 7        # skip between time points (in units of delta_t) 
interval = 2.  # time between frames in milliseconds
pendulum_anim.start_animation(skip, interval)

You can change the intial conditions in the code above. Let's add another pendulum with slightly different intial conditions and see what happens:

In [ ]:
plt.close('Pendulum animation') #close to avoid having to animations going at once and not plotting onto the same figure

In [ ]:
theta1_0p = theta1_0 + 0.01
# Create a pendulum animation instance. 
pendulum_anim2 = AnimationPendulumPlot(theta_vs_t=False, 
                                      physical_pend=True)
pendulum_anim2.t_pts_init(t_start, t_end, delta_t)
pendulum_anim2.add_pendulum(p1, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0)
pendulum_anim2.add_pendulum(p1, theta1_0p, theta2_0, theta1_dot_0, theta2_dot_0)
pendulum_anim2.plot_setup(plot_start, plot_end)
pendulum_anim2.start_animation(skip, interval)

In [ ]:
plt.close('Pendulum animation')

We see that a small perturbation results in very different behavior, which indicates chaos. Let's check this more closely.

## Chaos Checks

We first make a log plot of the difference in between the theta of one pendulum and another with slightly different intial conditions. We will arbitrarily to look at $\theta_2$, but we could look at either; as long as one coordinate behaves chaotically, our system is chaotic.

In [ ]:
#Initial conditions
theta1_0 = np.pi/2.
theta2_0 = np.pi/4.
theta1_dot_0 = 0.
theta2_dot_0 = 0.

theta2_0p = theta2_0 + 0.001

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 100.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  


# start the plot!
fig = plt.figure(figsize=(8,6))
overall_title = rf'Double pendulum from Lagrangian:  ' + \
                rf'L1 = L2 = m1 = m2 = g = 1'
fig.suptitle(overall_title, va='baseline')

theta1, theta2, theta1_dot, theta2_dot = p1.solve_ode(t_pts, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0)
theta1p, theta2p, theta1_dotp, theta2_dotp = p1.solve_ode(t_pts, theta1_0, theta2_0p, theta1_dot_0, theta2_dot_0)

diff = abs(theta2-theta2p)
diff2 = abs(theta1-theta1p)
# first plot: theta1 plot 
ax_a = fig.add_subplot(1,1,1)  
ax_a.set_ylabel(rf'$|\Delta\theta_2|$')
ax_a.set_xlabel(rf'$t$')


#ax_a.semilogy(t_pts, diff2, label=rf'$|\Delta\theta_1|$') 
ax_a.semilogy(t_pts, diff, label=rf'$|\Delta\theta_2|$') 
#ax_a.legend()

fig.tight_layout()

We see that the difference increases exponentially (looks linear on the log plot), indicating chaotic behavior. Let's also check some phase space plots. This time let's look at $\theta_1$ and $\theta_2$:

In [ ]:
#Initial conditions
theta1_0_l = [np.pi/2., 0., np.pi/2., np.pi]
theta2_0_l = [0., np.pi/2., np.pi/2., np.pi+0.01]
theta1_dot_0_l = [0., 0., 0., 0.]
theta2_dot_0_l = [0., np.pi/3., 0., 0.]


# start the plot!
fig = plt.figure(figsize=(8,12))
overall_title = rf'Double pendulum from Lagrangian:   ' + \
                rf'L1 = L2 = m1 = m2 = g = 1'
fig.suptitle(overall_title, va='baseline')

nrow = len(theta1_0_l)

#Solve and plot for each initial condition
for i in range(0, nrow):
    
    theta1_0 = theta1_0_l[i]
    theta2_0 = theta2_0_l[i]
    theta1_dot_0 = theta1_dot_0_l[i]
    theta2_dot_0 = theta2_dot_0_l[i]
    
    theta1, theta2, theta1_dot, theta2_dot = p1.solve_ode(t_pts, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0)
    
    # first plot: theta1 plot 
    ax_a = fig.add_subplot(nrow,2,(i+1)*2-1)
    ax_b = fig.add_subplot(nrow,2,(i+1)*2)                 


    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(theta1[start : stop], theta1_dot[start : stop], 
                axis_labels=state_space1_labels, 
                color='blue',
                title=rf'$\theta_1(0)={theta1_0:.3}, \theta_2(0)={theta2_0:.3}, $' + \
                rf'$\dot{{\theta_1}}(0)={theta1_dot_0:.3}, \dot{{\theta_2}}(0)={theta2_dot_0:.3}, $',
                ax=ax_a)       
    plot_y_vs_x(theta2[start : stop], theta2_dot[start : stop], 
                axis_labels=state_space2_labels, 
                color='red',
                title=None, 
                ax=ax_b)    

fig.tight_layout()


These don't look periodic, except for the second plot which seems to have hit on some sort of attractor, but it would be nice to see more filling of phase space. Let's run for a longer time and replot. We'll also cut off the first 100 seconds to make sure any chaotic looking behavior is not some sort of transient:

In [ ]:
# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 500.
delta_t = 0.01
t_pts = np.arange(t_start, t_end+delta_t, delta_t)  
t_start = 100.


# start the plot!
fig = plt.figure(figsize=(8,12))
overall_title = rf'Double pendulum from Lagrangian:   ' + \
                rf'L1 = L2 = m1 = m2 = g = 1'
fig.suptitle(overall_title, va='baseline')

nrow = len(theta1_0_l)

#Solve and plot for each initial condition
for i in range(0, nrow):
    
    theta1_0 = theta1_0_l[i]
    theta2_0 = theta2_0_l[i]
    theta1_dot_0 = theta1_dot_0_l[i]
    theta2_dot_0 = theta2_dot_0_l[i]
    
    theta1, theta2, theta1_dot, theta2_dot = p1.solve_ode(t_pts, theta1_0, theta2_0, theta1_dot_0, theta2_dot_0)
    
    # first plot: theta1 plot 
    ax_a = fig.add_subplot(nrow,2,(i+1)*2-1)
    ax_b = fig.add_subplot(nrow,2,(i+1)*2)                 


    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(theta1[start : stop], theta1_dot[start : stop], 
                axis_labels=state_space1_labels, 
                color='blue',
                title=rf'$\theta_1(0)={theta1_0:.3}, \theta_2(0)={theta2_0:.3}, $' + \
                rf'$\dot{{\theta_1}}(0)={theta1_dot_0:.3}, \dot{{\theta_2}}(0)={theta2_dot_0:.3}, $',
                ax=ax_a)       
    plot_y_vs_x(theta2[start : stop], theta2_dot[start : stop], 
                axis_labels=state_space2_labels, 
                color='red',
                title=None, 
                ax=ax_b)    

fig.tight_layout()

Here we have pretty convincing filling of phase space, but interestingly it seems our attractors for the second set of initial conditions have persisted. Nonetheless, at this point we conclude the motion is chaotic for at least some initial conditions.